In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import pandas as pd

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom, recgeom_indivFaces

In [2]:
# 3 layer model - USER DEFINED
expname = 'homoRB'
outpath = '../../FDModelling/HomoForDAS/'

# Rock Properties - subsurface models

In [3]:
dimlims = 0.4 # 1m cubed
dx = dy = dz = 0.01  # 1cm
nx = ny = nz = int(dimlims/dx)
# dimlims = nx*dx

# Subsurface properties
vp = 5260  # m/s
vs = 3260  # m/s
rho = 2590;  # kg/m3


In [4]:
subsurface_properties = [vp, vs, rho]
vp_mod, vs_mod, rho_mod = make_homo_model(dimlims,
                                            dx,
                                            subsurface_properties,
                                            pad=False,
                                            npad=0,
                                            freesurface=False
                                            )

# Write as binary
os.makedirs(os.path.join(outpath,'inputs/model/'), exist_ok=True)
vp_mod.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.vp'%expname))#,format='<f')
vs_mod.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.vs'%expname))  #,format='<f')
rho_mod.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.rho'%expname))  #,format='<f')

# Acquisition Geometry - Receivers & Source

In [5]:
# Source 
sx, sy, sz = 0.2, 0.2, 0.2
td = 0.  # ignition time
fc = 25000  # central frequency
amp = 1.  # amplitude
string_list = [str(i) for i in [sx, sz, sy, td, fc, amp]]
src_string = '\t'.join(string_list)
print(src_string)

text_file = open(os.path.join(outpath,'inputs/sources.dat'), "wt")
text_file.writelines(src_string)
text_file.close()

0.2	0.2	0.2	0.0	25000	1.0


In [6]:
# ALL FACES
nrperface = 40

# don't need to change below
nxyz = [nx, ny, nz]
dxyz = [dx, dy, dz]
gridded_recs, nr, rec_string = recgeom(nxyz, dxyz, nrperfc_xyz=[nrperface], recbuf_gps=1)
np.savetxt(os.path.join(outpath,'inputs/receiver.dat'),
           gridded_recs.T.astype(np.float32), 
           fmt='%.2f')

# BUILD JSON FOR SOFI

In [7]:
# mod params
n_xzy = vp_mod.transpose(0,2,1).shape
d_xzy = [dx, dz, dy]

# time params (in seconds!)
dt = 1e-7
tdur = 0.001
tsnap_params = [dt, tdur-dt, dt*2]
smgrm_ndt = 1
SOFIjsonfilename = os.path.join(outpath,'SOFI3D_%s.json'%expname)

sofimaster = '~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/'
num_procs = 8
sofi_param_file = "SOFI3D_%s.json"%expname
shellfilename = os.path.join(outpath,'runSOFI3D_%s.sh'%expname)


In [8]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings()

monitor_strs = get_monitor_str(tsnap_params, smgrm_ndt, expname, sbsmp_xyz=1)
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xzy, d_xzy,expname)
bndry_str = get_boundary_str(fs=False, npad=0,cpml=False)
src_str = get_source_str()

write_SOFIjsonParams = write_SOFIjsonParams(default_strs, 
                                            monitor_strs,  
                                            t_str, 
                                            mod_strs, 
                                            src_str, 
                                            bndry_str, 
                                            SOFIjsonfilename)
runscript = write_SOFIrunScript(sofimaster, 
                                num_procs, 
                                sofi_param_file, 
                                shellfilename)

In [9]:
vp_mod.transpose(0,2,1).shape

(40, 40, 40)